# A Python Module for Mobility Determination of Parallel Manipulators Using Screw Theory

<br><br>

### Adriyan

<br>

**Department of Mechanical Engineering**  
**Sekolah Tinggi Teknologi Nasional, Jambi, 36125, Indonesia**
<br><br>
&copy; Adriyan, 2019
<br><br>

---

<font color="navy">
This material is used as a part of a research article which entitled <strong><em>A Python Module for Mobility Determination of Parallel Manipulators Using Screw Theory</em></strong> which published in Journal METAL, Andalas University.
    <br><br>
Material appears in this notebook is released under 3-clause BSD License. 
    
**If you found this notebook and/or the <code>pyScrew4Mobility</code> package is worth for your research or publications please cites the paper.**
</font>

---
<br>

In [1]:
from IPython.display import HTML, display_png
# with open("jlstyle.css", "r") as fs: fs=fs.read() # If you run on Jupyter Lab
with open("nbstyle.css", "r") as fs: fs=fs.read() # If you run on Jupyter Notebook
HTML(fs)

## 1. A Grübler-Kutzbach Mobility

Grübler-Kutzbach mobility formulas:  

$$
M = \kappa \cdot \left( n - j - 1\right) + \sum_{i}{f_i}
$$

where $\kappa$ is equal to 3 and 6 for planar and spatial manipulators, respectively. $n$ and $j$ denote the number of links and joints in the mechanisms/manipulators, respectively. While, $\sum_{i}{f_i}$ is the number of motion allowed by each joint that constructs the mechanisms/manipulators.

In [2]:
def Mobility_GK(n, j, Sigma_fi, kappa=6):
    return kappa * (n - j - 1) + Sigma_fi

## 2. `pyScrew4Mobility` Module: the implementation of screw theory for mobility determination of parallel manipulators using sympy

Currently, `pyScrew4Mobility` Module is constructed by three python classes:
1. A class of `Screw` which responsible to construct a screw or a screw system.  
2. A class of `ScrewSystem`  which responsible for algebraic calculation of the screw or the screw system, such as orthogonal product and reciprocal screw.  
3. A class of `Mobility` which applied to calculate mobility of parallel manipulators for given screw system of its each limb.   

The `pyScrew4Mobility Module` is released under 3-Clause BSD (Berkeley Software Disribution) License.  

To use `pyScrew4Mobility Module` using this (Jupyter) notebook, one must put the the file `pyScrew4Mobility.py` in the same directory of this notebook and use the IPython magic as given below before importing the screw library.

In [3]:
%load_ext autoreload
%autoreload 2
import pyScrew4Mobility as s4m

## 3. The Application of `pyScrew4Mobility` Package to Four Well-known Parallel Manipulators

The `pyScrew4Mobility` package is applied to determine the mobility of four well-known parallel manipulators, namely:
1. **Tripteron: a 3-PRRR parallel manipulator**
2. **A star-like parallel manipulator 3-PR(Pa)R**
3. **A 4-PRRU parallel manipulator**, Quadrupteron (PRRR/3-PRRU) is the special case.
4. **A 6-UPS parallel manipulator**

In [4]:
import sympy as sym
from sympy.physics import mechanics as mec
from sympy.physics import matrices as mat

In [5]:
sym.init_printing(use_latex="mathjax")

### 3.1. A 3-PRRR Parallel Manipulator

The geometry of manipulator

<img src="./img/1_PM_3PRRR.PNG" width="450" />

**Examining the mobility with Grübler-Kutzbach mobility formula**

In [6]:
n = 1 + 3 * 3 + 1
j = 3 * 4
Sigma_fi = 3 * 4
print("Mobility of 3-PRRR using Grübler-Kutzbach mobility formulas: " \
      + str(Mobility_GK(n, j, Sigma_fi)))

Mobility of 3-PRRR using Grübler-Kutzbach mobility formulas: 0


**The zeroth step:** A user defined for ${\bf{r}}_{ij}$, ${\bf{s}}_{ij}$ and $h_{ij}$ of 3-PRRR PM.

In [7]:
def PM_3PRRR():
    # Number of limbs
    n_limbs = 3 
    # Declaring sympy symbol for its kinematic parameter
    a, b, Ly, Lz, r = sym.symbols("a b L_y L_z r") 
    # Joint displacement and/or joint angular displacement
    act_spc = sym.symbols("d_{1:"+str(n_limbs+1)+"}")
    thetas = sym.symbols("\\theta_{{1:"+str(n_limbs+1)+"}{2:4}}")
    # The unit vector of a unit screw pointing at for each limb
    _s = [4 * [mat.Matrix([1, 0, 0]),],
          4 * [mat.Matrix([0, 1, 0]),],
          4 * [mat.Matrix([0, 0, 1])]]
    # pitch of the each joint for every limb
    _h = [sym.oo,] + 3 * [0,]    
    # A fixed reference of frame O-XYZ
    G = mec.ReferenceFrame("G")    
    # Position vector of OA_i
    rOA = [mat.Matrix([act_spc[0], Ly, 0]),
           mat.Matrix([0, act_spc[1], Lz]),
           mat.Matrix([0, 0, act_spc[2]])]
    # An auxiliary vector for determining position vector AB and BC
    _ = [mat.Matrix([0, 1, 0]),
         mat.Matrix([0, 0, 1]),
         mat.Matrix([1, 0, 0])]
    # Within this loop, it is constructed all parameters required for 
    # mobility calculation
    list_of_parameters = [] # Variable for containing all parameters 
                            # required for mobility calculation
    ii = 0
    for _1, _2, _3 in zip(thetas[::2], thetas[1::2], [G.x, G.y, G.z]):
        # Moving reference of frame
        H = G.orientnew("H"+str(ii+1), "Axis", [_1, _3])
        K = G.orientnew("K"+str(ii+1), "Axis", [_2, _3])
        # Position vector AB and BC in fixed reference of frame
        rAB = G.dcm(H) * (a * _[ii])
        rBC = G.dcm(K) * (b * _[ii])
        # Contruct all parameters for each limb
        s_limb = []
        for jj in range(len(_s[ii])):
            _r = rOA[ii]
            if jj == 2:
                _r += rAB
            elif jj == 3:
                _r += rAB + rBC
            s_limb.append([_s[ii][jj], _r, _h[jj]])
        list_of_parameters.append(s_limb)
        ii += 1
    return list_of_parameters


**The first to fourth step:**

In [8]:
Mobility_PM_3PRRR = s4m.ManipulatorsMobility(PM_3PRRR(), "A 3-PRRR Parallel Manipulator")

In [9]:
Mobility_PM_3PRRR.Mobility

{'Number': 3, 'Type': '3T', 'Direction': ['Tx', 'Ty', 'Tz']}

**Displaying the result from the first to fourth step:** 

In [10]:
def all_screw_system(obj):
    print("Limb Twist System")
    print("=================")
    for ii, item in enumerate(obj.LimbTwistSystem):
        print("Limb: ", ii+1)
        display(item.screw_system_list)

    print("\n\nLimb Wrench System")
    print("==================")
    for ii, item in enumerate(obj.LimbWrenchSystem):
        print("Limb: ", ii+1)
        display(item.screw_system_list)

    print("\n\nPlatform Wrench System")
    print("======================")
    display(obj.PlatformWrenchSystem.screw_system_list)

    print("\n\nPlatform Twist System")
    print("=====================")
    display(obj.PlatformTwistSystem.screw_system_list)

In [11]:
all_screw_system(Mobility_PM_3PRRR)

Limb Twist System
Limb:  1


⎡⎡0⎤  ⎡ 1  ⎤  ⎡              1              ⎤  ⎡                          1   
⎢⎢ ⎥  ⎢    ⎥  ⎢                             ⎥  ⎢                              
⎢⎢0⎥  ⎢ 0  ⎥  ⎢              0              ⎥  ⎢                          0   
⎢⎢ ⎥  ⎢    ⎥  ⎢                             ⎥  ⎢                              
⎢⎢0⎥  ⎢ 0  ⎥  ⎢              0              ⎥  ⎢                          0   
⎢⎢ ⎥, ⎢    ⎥, ⎢                             ⎥, ⎢                              
⎢⎢1⎥  ⎢ 0  ⎥  ⎢              0              ⎥  ⎢                          0   
⎢⎢ ⎥  ⎢    ⎥  ⎢                             ⎥  ⎢                              
⎢⎢0⎥  ⎢ 0  ⎥  ⎢   a⋅sin(\theta_{{1}{2}})    ⎥  ⎢   a⋅sin(\theta_{{1}{2}}) + b⋅
⎢⎢ ⎥  ⎢    ⎥  ⎢                             ⎥  ⎢                              
⎣⎣0⎦  ⎣-L_y⎦  ⎣-L_y - a⋅cos(\theta_{{1}{2}})⎦  ⎣-L_y - a⋅cos(\theta_{{1}{2}}) 

                        ⎤⎤
                        ⎥⎥
                        ⎥⎥
                        ⎥⎥
                      

Limb:  2


⎡⎡0⎤  ⎡ 0  ⎤  ⎡              0              ⎤  ⎡                          0   
⎢⎢ ⎥  ⎢    ⎥  ⎢                             ⎥  ⎢                              
⎢⎢0⎥  ⎢ 1  ⎥  ⎢              1              ⎥  ⎢                          1   
⎢⎢ ⎥  ⎢    ⎥  ⎢                             ⎥  ⎢                              
⎢⎢0⎥  ⎢ 0  ⎥  ⎢              0              ⎥  ⎢                          0   
⎢⎢ ⎥, ⎢    ⎥, ⎢                             ⎥, ⎢                              
⎢⎢0⎥  ⎢-L_z⎥  ⎢-L_z - a⋅cos(\theta_{{2}{2}})⎥  ⎢-L_z - a⋅cos(\theta_{{2}{2}}) 
⎢⎢ ⎥  ⎢    ⎥  ⎢                             ⎥  ⎢                              
⎢⎢1⎥  ⎢ 0  ⎥  ⎢              0              ⎥  ⎢                          0   
⎢⎢ ⎥  ⎢    ⎥  ⎢                             ⎥  ⎢                              
⎣⎣0⎦  ⎣ 0  ⎦  ⎣   a⋅sin(\theta_{{2}{2}})    ⎦  ⎣   a⋅sin(\theta_{{2}{2}}) + b⋅

                        ⎤⎤
                        ⎥⎥
                        ⎥⎥
                        ⎥⎥
                      

Limb:  3


...

⎡⎡0⎤  ⎡0⎤⎤
⎢⎢ ⎥  ⎢ ⎥⎥
⎢⎢0⎥  ⎢0⎥⎥
⎢⎢ ⎥  ⎢ ⎥⎥
⎢⎢0⎥  ⎢0⎥⎥
⎢⎢ ⎥, ⎢ ⎥⎥
⎢⎢1⎥  ⎢0⎥⎥
⎢⎢ ⎥  ⎢ ⎥⎥
⎢⎢0⎥  ⎢1⎥⎥
⎢⎢ ⎥  ⎢ ⎥⎥
⎣⎣0⎦  ⎣0⎦⎦



Platform Wrench System


⎡⎡0⎤  ⎡0⎤  ⎡0⎤⎤
⎢⎢ ⎥  ⎢ ⎥  ⎢ ⎥⎥
⎢⎢0⎥  ⎢0⎥  ⎢0⎥⎥
⎢⎢ ⎥  ⎢ ⎥  ⎢ ⎥⎥
⎢⎢0⎥  ⎢0⎥  ⎢0⎥⎥
⎢⎢ ⎥, ⎢ ⎥, ⎢ ⎥⎥
⎢⎢0⎥  ⎢0⎥  ⎢1⎥⎥
⎢⎢ ⎥  ⎢ ⎥  ⎢ ⎥⎥
⎢⎢1⎥  ⎢0⎥  ⎢0⎥⎥
⎢⎢ ⎥  ⎢ ⎥  ⎢ ⎥⎥
⎣⎣0⎦  ⎣1⎦  ⎣0⎦⎦



Platform Twist System


⎡⎡0⎤  ⎡0⎤  ⎡0⎤⎤
⎢⎢ ⎥  ⎢ ⎥  ⎢ ⎥⎥
⎢⎢0⎥  ⎢0⎥  ⎢0⎥⎥
⎢⎢ ⎥  ⎢ ⎥  ⎢ ⎥⎥
⎢⎢0⎥  ⎢0⎥  ⎢0⎥⎥
⎢⎢ ⎥, ⎢ ⎥, ⎢ ⎥⎥
⎢⎢1⎥  ⎢0⎥  ⎢0⎥⎥
⎢⎢ ⎥  ⎢ ⎥  ⎢ ⎥⎥
⎢⎢0⎥  ⎢1⎥  ⎢0⎥⎥
⎢⎢ ⎥  ⎢ ⎥  ⎢ ⎥⎥
⎣⎣0⎦  ⎣0⎦  ⎣1⎦⎦

### 3.2. A 3-PR(Pa)R Parallel Manipulator

The geometry of manipulator

<img src="./img/2_PM_3PRPaR.PNG" width="512" />

**Examining the mobility with Grübler-Kutzbach mobility formula**

In [12]:
n = 1 + 3 * 5 + 1
j = 3 * 7
Sigma_fi = 3 * 7
print("Mobility of 3-PR(Pa)R using Grübler-Kutzbach mobility formulas: " \
      + str(Mobility_GK(n, j, Sigma_fi)))

Mobility of 3-PR(Pa)R using Grübler-Kutzbach mobility formulas: -9


**The zeroth step:** A user defined for ${\bf{r}}_{ij}$, ${\bf{s}}_{ij}$ and $h_{ij}$ of 3-PR(Pa)R PM.

In [13]:
def PM_3PRPaR():
    # Number of limbs
    n_limbs = 3
    # Declaring sympy symbol for its kinematic parameter
    L, r = sym.symbols("L r")
    a, b = sym.symbols("a b")
    # Joint displacement and/or joint angular displacement
    act_spc = sym.symbols("d_{1:"+str(n_limbs+1)+"}")
    lambdas = sym.symbols("\\lambda_{1:"+str(n_limbs+1)+"}")
    thetas = sym.symbols("\\theta_{{1:"+str(n_limbs+1)+"}{2:6}}")
    lambdas_subs = dict(zip(lambdas, [0, 2*sym.pi/3, 4*sym.pi/3]))
    # A fixed reference of frame O-XYZ
    G = mec.ReferenceFrame("G")
    # Within this loop, it is constructed all parameters required for 
    # mobility calculation
    list_of_parameters = [] # Variable for containing all parameters 
                            # required for mobility calculation
    kk = len(thetas)//n_limbs
    for ii in range(n_limbs):
        # Moving reference of frame
        H = G.orientnew("H"+str(ii+1), "Axis", [lambdas[ii].subs(lambdas_subs), G.z])
        rOA0 = mat.Matrix([act_spc[ii], 0, 0])
        rOA1 = mat.Matrix([act_spc[ii] - b/2, 0, 0])
        rOA2 = mat.Matrix([act_spc[ii] + b/2, 0, 0])
        K = G.orientnew("K"+str(ii+1), "Axis", [thetas[ii*kk], G.x])
        N = G.orientnew("N"+str(ii+1), "Body", [*thetas[ii*kk:ii*kk+2], 0], "XYZ")
        # Position vector A0B0, A1B1, and A2B2 in fixed reference of frame
        rA0B0, rA1B1, rA2B2 = 3 * [G.dcm(N) * mat.Matrix([L, 0, 0]),]
        # The unit vector of a unit screw pointing at for each limb
        _s = 2 * [mat.Matrix([1, 0, 0]),] \
             + 4 * [mat.Matrix([0, 1, 0]),] \
             + [mat.Matrix([1, 0, 0]),]
        # pitch of the each joint for every limb
        _h = [sym.oo,] + 6 * [0,]
        # Contruct all parameters for each limb
        s_limb = []
        for jj, item in enumerate(_s):
            _r = rOA0
            if jj in [2, 3, 4, 5, 6]:
                item = G.dcm(K) * item
                if jj == 2:
                    _r = rOA1
                elif jj == 3:
                    _r = rOA2
                elif jj == 4:
                    _r = rOA1 + rA1B1
                elif jj == 5:
                    _r = rOA2 + rA2B2
                elif jj == 6:
                    _r = rOA0 + rA0B0 
            s_limb.append([G.dcm(H) * item, G.dcm(H) * _r, _h[jj]])  
        list_of_parameters.append(s_limb)
    return list_of_parameters


**The first to fourth step:**

In [14]:
Mobility_PM_3PRPaR = s4m.ManipulatorsMobility(PM_3PRPaR(), "A 3-PR(Pa)R Parallel Manipulator")

In [15]:
Mobility_PM_3PRPaR.Mobility

{'Number': 3, 'Type': '3T', 'Direction': ['Tx', 'Ty', 'Tz']}

**Displaying the result from the first to fourth step:** 

In [16]:
all_screw_system(Mobility_PM_3PRPaR)

Limb Twist System
Limb:  1


⎡          ⎡                0                 ⎤  ⎡                0           
⎢          ⎢                                  ⎥  ⎢                            
⎢⎡0⎤  ⎡1⎤  ⎢       cos(\theta_{{1}{2}})       ⎥  ⎢      cos(\theta_{{1}{2}})  
⎢⎢ ⎥  ⎢ ⎥  ⎢                                  ⎥  ⎢                            
⎢⎢0⎥  ⎢0⎥  ⎢       sin(\theta_{{1}{2}})       ⎥  ⎢      sin(\theta_{{1}{2}})  
⎢⎢ ⎥  ⎢ ⎥  ⎢                                  ⎥  ⎢                            
⎢⎢0⎥  ⎢0⎥  ⎢                0                 ⎥  ⎢                0           
⎢⎢ ⎥, ⎢ ⎥, ⎢                                  ⎥, ⎢                            
⎢⎢1⎥  ⎢0⎥  ⎢ ⎛b        ⎞                      ⎥  ⎢ ⎛b        ⎞                
⎢⎢ ⎥  ⎢ ⎥  ⎢ ⎜─ - d_{1}⎟⋅sin(\theta_{{1}{2}}) ⎥  ⎢-⎜─ + d_{1}⎟⋅sin(\theta_{{1}
⎢⎢0⎥  ⎢0⎥  ⎢ ⎝2        ⎠                      ⎥  ⎢ ⎝2        ⎠                
⎢⎢ ⎥  ⎢ ⎥  ⎢                                  ⎥  ⎢                            
⎢⎣0⎦  ⎣0⎦  ⎢⎛  b        ⎞                     ⎥  ⎢⎛b

Limb:  2


⎡                ⎡      -√3⋅cos(\theta_{{2}{2}})        ⎤  ⎡      -√3⋅cos(\the
⎢                ⎢      ─────────────────────────       ⎥  ⎢      ────────────
⎢                ⎢                  2                   ⎥  ⎢                  
⎢                ⎢                                      ⎥  ⎢                  
⎢⎡ 0  ⎤  ⎡-1/2⎤  ⎢        -cos(\theta_{{2}{2}})         ⎥  ⎢       -cos(\theta
⎢⎢    ⎥  ⎢    ⎥  ⎢        ──────────────────────        ⎥  ⎢       ───────────
⎢⎢ 0  ⎥  ⎢ √3 ⎥  ⎢                  2                   ⎥  ⎢                 2
⎢⎢    ⎥  ⎢ ── ⎥  ⎢                                      ⎥  ⎢                  
⎢⎢ 0  ⎥  ⎢ 2  ⎥  ⎢         sin(\theta_{{2}{2}})         ⎥  ⎢        sin(\theta
⎢⎢    ⎥  ⎢    ⎥  ⎢                                      ⎥  ⎢                  
⎢⎢-1/2⎥, ⎢ 0  ⎥, ⎢√3⋅(-b + 2⋅d_{2})⋅sin(\theta_{{2}{2}})⎥, ⎢√3⋅(b + 2⋅d_{2})⋅s
⎢⎢    ⎥  ⎢    ⎥  ⎢──────────────────────────────────────⎥  ⎢──────────────────
⎢⎢ √3 ⎥  ⎢ 0  ⎥  ⎢                  4               

Limb:  3


...

⎡⎡            0            ⎤⎤
⎢⎢                         ⎥⎥
⎢⎢            0            ⎥⎥
⎢⎢                         ⎥⎥
⎢⎢            0            ⎥⎥
⎢⎢                         ⎥⎥
⎢⎢-√3⋅tan(\theta_{{3}{2}}) ⎥⎥
⎢⎢─────────────────────────⎥⎥
⎢⎢            2            ⎥⎥
⎢⎢                         ⎥⎥
⎢⎢  tan(\theta_{{3}{2}})   ⎥⎥
⎢⎢  ────────────────────   ⎥⎥
⎢⎢           2             ⎥⎥
⎢⎢                         ⎥⎥
⎣⎣            1            ⎦⎦



Platform Wrench System


⎡                         ⎡           0           ⎤  ⎡            0           
⎢                         ⎢                       ⎥  ⎢                        
⎢⎡          0          ⎤  ⎢           0           ⎥  ⎢            0           
⎢⎢                     ⎥  ⎢                       ⎥  ⎢                        
⎢⎢          0          ⎥  ⎢           0           ⎥  ⎢            0           
⎢⎢                     ⎥  ⎢                       ⎥  ⎢                        
⎢⎢          0          ⎥  ⎢√3⋅tan(\theta_{{2}{2}})⎥  ⎢-√3⋅tan(\theta_{{3}{2}})
⎢⎢                     ⎥, ⎢───────────────────────⎥, ⎢────────────────────────
⎢⎢          0          ⎥  ⎢           2           ⎥  ⎢            2           
⎢⎢                     ⎥  ⎢                       ⎥  ⎢                        
⎢⎢-tan(\theta_{{1}{2}})⎥  ⎢ tan(\theta_{{2}{2}})  ⎥  ⎢  tan(\theta_{{3}{2}})  
⎢⎢                     ⎥  ⎢ ────────────────────  ⎥  ⎢  ────────────────────  
⎢⎣          1          ⎦  ⎢          2            ⎥ 



Platform Twist System


⎡⎡0⎤  ⎡0⎤  ⎡0⎤⎤
⎢⎢ ⎥  ⎢ ⎥  ⎢ ⎥⎥
⎢⎢0⎥  ⎢0⎥  ⎢0⎥⎥
⎢⎢ ⎥  ⎢ ⎥  ⎢ ⎥⎥
⎢⎢0⎥  ⎢0⎥  ⎢0⎥⎥
⎢⎢ ⎥, ⎢ ⎥, ⎢ ⎥⎥
⎢⎢1⎥  ⎢0⎥  ⎢0⎥⎥
⎢⎢ ⎥  ⎢ ⎥  ⎢ ⎥⎥
⎢⎢0⎥  ⎢1⎥  ⎢0⎥⎥
⎢⎢ ⎥  ⎢ ⎥  ⎢ ⎥⎥
⎣⎣0⎦  ⎣0⎦  ⎣1⎦⎦

### 3.3. 4-PRRU Parallel Manipulators

The geometry of manipulator

<img src="./img/5_PM_4PRRU.png" width="512" />

**Examining the mobility with Grübler-Kutzbach mobility formula**

In [17]:
n = 1 + 4 * 3 + 1
j = 4 * 4
Sigma_fi = 4 * (3 + 2)
print("Mobility of 4-PRRU using Grübler-Kutzbach mobility formulas: " \
      + str(Mobility_GK(n, j, Sigma_fi)))

Mobility of 4-PRRU using Grübler-Kutzbach mobility formulas: 2


**The zeroth step:** A user defined for ${\bf{r}}_{ij}$, ${\bf{s}}_{ij}$ and $h_{ij}$ of 4-PRRU PM.

In [18]:
def PM_4PRRU():
    # Number of limbs
    n_limbs = 4
    # Declaring sympy symbol for its kinematic parameter
    R, a, b, alpha = sym.symbols("R a b \\alpha")
    # Joint displacement and/or joint angular displacement
    act_spc = sym.symbols("d_{1:"+str(n_limbs+1)+"}")
    thetas = sym.symbols("\\theta_{{1:"+str(n_limbs+1)+"}{2:6}}")
    lambdas = sym.symbols("\\lambda_{1:"+str(n_limbs+1)+"}")
    lambdas_subs = dict(zip(lambdas, [0, sym.pi/2, sym.pi, 3*sym.pi/2]))
    # The unit vector of a unit screw pointing at for each limb
    _s = [mat.Matrix([0, 0, 1]),
          mat.Matrix([0, 1, 0]),
          mat.Matrix([0, 1, 0]),
          mat.Matrix([0, 1, 0]),
          mat.Matrix([0, 0, 1])]
    # pitch of the each joint for every limb
    _h = [sym.oo,] + 4 * [0,]    
    # A fixed reference of frame O-XYZ
    G = mec.ReferenceFrame("G")
    # Within this loop, it is constructed all parameters required for 
    # mobility calculation
    list_of_parameters = [] # Variable for containing all parameters 
                            # required for mobility calculation
    kk = len(thetas)//n_limbs
    for ii in range(n_limbs):
        # Moving reference of frame
        H = G.orientnew("H"+str(ii+1), "Axis", 
                        [lambdas[ii].subs(lambdas_subs), G.z])
        K = G.orientnew("K"+str(ii+1), "Axis", [alpha, G.x])
        N = G.orientnew("N"+str(ii+1), "Axis",
                        [thetas[ii*kk], K.y])
        Q = G.orientnew("Q"+str(ii+1), "Axis",
                        [thetas[ii*kk+1], K.y])
        # Position vector OA, AB, and BC in fixed reference of frame
        rOA = G.dcm(H) * mat.Matrix([R, 0, act_spc[ii]])
        rAB = G.dcm(H) * (G.dcm(K) * (G.dcm(N) * mat.Matrix([a, 0, 0])))
        rBC = G.dcm(H) * (G.dcm(N) * (G.dcm(N) * mat.Matrix([b, 0, 0])))
        # Contruct all parameters for each limb
        s_limb = []
        for jj, item in enumerate(_s):
            if jj in [1, 2, 3]:
                item = G.dcm(K) * item
            item = G.dcm(H) * item
            if jj in [0, 1]:
                _r = rOA
            elif jj == 2:
                _r = rOA + rAB
            elif jj in [3, 4]:
                _r = rOA + rAB + rBC
            s_limb.append([item, _r, _h[jj]])      
        list_of_parameters.append(s_limb)
    return list_of_parameters


**The first to fourth step:**

In [19]:
Mobility_PM_4PRRU = s4m.ManipulatorsMobility(PM_4PRRU(), "4-PRRU Parallel Manipulators")

**Displaying the result from the first to fourth step:** 

In [20]:
all_screw_system(Mobility_PM_4PRRU)

Limb Twist System
Limb:  1


⎡                                                                           ⎡ 
⎢⎡0⎤  ⎡        0         ⎤  ⎡                     0                      ⎤  ⎢ 
⎢⎢ ⎥  ⎢                  ⎥  ⎢                                            ⎥  ⎢ 
⎢⎢0⎥  ⎢   cos(\alpha)    ⎥  ⎢                cos(\alpha)                 ⎥  ⎢ 
⎢⎢ ⎥  ⎢                  ⎥  ⎢                                            ⎥  ⎢ 
⎢⎢0⎥  ⎢   sin(\alpha)    ⎥  ⎢                sin(\alpha)                 ⎥  ⎢ 
⎢⎢ ⎥, ⎢                  ⎥, ⎢                                            ⎥, ⎢a
⎢⎢0⎥  ⎢-d_{1}⋅cos(\alpha)⎥  ⎢(a⋅sin(\theta_{{1}{2}}) - d_{1})⋅cos(\alpha)⎥  ⎢ 
⎢⎢ ⎥  ⎢                  ⎥  ⎢                                            ⎥  ⎢ 
⎢⎢0⎥  ⎢  -R⋅sin(\alpha)  ⎥  ⎢ -(R + a⋅cos(\theta_{{1}{2}}))⋅sin(\alpha)  ⎥  ⎢ 
⎢⎢ ⎥  ⎢                  ⎥  ⎢                                            ⎥  ⎢ 
⎢⎣1⎦  ⎣  R⋅cos(\alpha)   ⎦  ⎣  (R + a⋅cos(\theta_{{1}{2}}))⋅cos(\alpha)  ⎦  ⎢ 
⎣                                                   

Limb:  2


⎡                                                                           ⎡ 
⎢⎡0⎤  ⎡   -cos(\alpha)   ⎤  ⎡                -cos(\alpha)                ⎤  ⎢ 
⎢⎢ ⎥  ⎢                  ⎥  ⎢                                            ⎥  ⎢ 
⎢⎢0⎥  ⎢        0         ⎥  ⎢                     0                      ⎥  ⎢ 
⎢⎢ ⎥  ⎢                  ⎥  ⎢                                            ⎥  ⎢ 
⎢⎢0⎥  ⎢   sin(\alpha)    ⎥  ⎢                sin(\alpha)                 ⎥  ⎢ 
⎢⎢ ⎥, ⎢                  ⎥, ⎢                                            ⎥, ⎢ 
⎢⎢0⎥  ⎢  R⋅sin(\alpha)   ⎥  ⎢  (R + a⋅cos(\theta_{{2}{2}}))⋅sin(\alpha)  ⎥  ⎢ 
⎢⎢ ⎥  ⎢                  ⎥  ⎢                                            ⎥  ⎢ 
⎢⎢0⎥  ⎢-d_{2}⋅cos(\alpha)⎥  ⎢(a⋅sin(\theta_{{2}{2}}) - d_{2})⋅cos(\alpha)⎥  ⎢a
⎢⎢ ⎥  ⎢                  ⎥  ⎢                                            ⎥  ⎢ 
⎢⎣1⎦  ⎣  R⋅cos(\alpha)   ⎦  ⎣  (R + a⋅cos(\theta_{{2}{2}}))⋅cos(\alpha)  ⎦  ⎢ 
⎣                                                   

Limb:  3


...

⎡⎡0⎤⎤
⎢⎢ ⎥⎥
⎢⎢0⎥⎥
⎢⎢ ⎥⎥
⎢⎢0⎥⎥
⎢⎢ ⎥⎥
⎢⎢0⎥⎥
⎢⎢ ⎥⎥
⎢⎢1⎥⎥
⎢⎢ ⎥⎥
⎣⎣0⎦⎦



Platform Wrench System


⎡⎡0⎤  ⎡0⎤⎤
⎢⎢ ⎥  ⎢ ⎥⎥
⎢⎢0⎥  ⎢0⎥⎥
⎢⎢ ⎥  ⎢ ⎥⎥
⎢⎢0⎥  ⎢0⎥⎥
⎢⎢ ⎥, ⎢ ⎥⎥
⎢⎢1⎥  ⎢0⎥⎥
⎢⎢ ⎥  ⎢ ⎥⎥
⎢⎢0⎥  ⎢1⎥⎥
⎢⎢ ⎥  ⎢ ⎥⎥
⎣⎣0⎦  ⎣0⎦⎦



Platform Twist System


⎡⎡0⎤  ⎡0⎤  ⎡0⎤  ⎡0⎤⎤
⎢⎢ ⎥  ⎢ ⎥  ⎢ ⎥  ⎢ ⎥⎥
⎢⎢0⎥  ⎢0⎥  ⎢0⎥  ⎢0⎥⎥
⎢⎢ ⎥  ⎢ ⎥  ⎢ ⎥  ⎢ ⎥⎥
⎢⎢0⎥  ⎢0⎥  ⎢0⎥  ⎢1⎥⎥
⎢⎢ ⎥, ⎢ ⎥, ⎢ ⎥, ⎢ ⎥⎥
⎢⎢1⎥  ⎢0⎥  ⎢0⎥  ⎢0⎥⎥
⎢⎢ ⎥  ⎢ ⎥  ⎢ ⎥  ⎢ ⎥⎥
⎢⎢0⎥  ⎢1⎥  ⎢0⎥  ⎢0⎥⎥
⎢⎢ ⎥  ⎢ ⎥  ⎢ ⎥  ⎢ ⎥⎥
⎣⎣0⎦  ⎣0⎦  ⎣1⎦  ⎣0⎦⎦

### 3.4. 6-UPS Parallel Manipulators

The geometry of manipulator

<img src="./img/6_PM_6UPS.PNG" width="512" />

**Examining the mobility with Grübler-Kutzbach mobility formula**

In [21]:
n = 1 + 6 * 2 + 1
j = 6 * 3
Sigma_fi = 6 * (2 + 1 + 3)
print("Mobility of 6-UPS using Grübler-Kutzbach mobility formulas: " \
      + str(Mobility_GK(n, j, Sigma_fi)))

Mobility of 6-UPS using Grübler-Kutzbach mobility formulas: 6


**The zeroth step:** A user defined for ${\bf{r}}_{ij}$, ${\bf{s}}_{ij}$ and $h_{ij}$ of 6-UPS PM.

In [22]:
def PM_6UPS():
    # Number of limbs
    n_limbs = 6
    # Declaring sympy symbol for its kinematic parameter
    R, r = sym.symbols("R r")
    # Joint displacement and/or joint angular displacement
    act_spc = sym.symbols("d_{1:"+str(n_limbs+1)+"}")
    thetas = sym.symbols("\\theta_{{1:"+str(n_limbs+1)+"}{1:6}}")
    lambdas = sym.symbols("\\lambda_{1:"+str(n_limbs+1)+"}")
    lambdas_subs = dict(zip(lambdas, [ii*sym.pi/3 for ii in range(n_limbs)]))
    # The unit vector of a unit screw pointing at for each limb
    _s = [mat.Matrix([1, 0, 0]),
          mat.Matrix([0, 1, 0]),
          mat.Matrix([0, 0, 1]),
          mat.Matrix([1, 0, 0]),
          mat.Matrix([0, 1, 0]),
          mat.Matrix([0, 0, 1])]
    # pitch of the each joint for every limb
    _h = 2 * [0,] + [sym.oo,] + 3 * [0,]
    # A fixed reference of frame O-XYZ
    G = mec.ReferenceFrame("G")
    # Within this loop, it is constructed all parameters required for 
    # mobility calculation
    list_of_parameters = [] # Variable for containing all parameters 
                            # required for mobility calculation
    kk = len(thetas)//n_limbs
    for ii in range(n_limbs):
        # Moving reference of frame
        H = G.orientnew("H"+str(ii+1), "Axis", 
                        [lambdas[ii].subs(lambdas_subs), G.z])
        K = G.orientnew("K"+str(ii+1), "Axis", [thetas[ii*kk], G.x])
        N = K.orientnew("N"+str(ii+1), "Axis", [thetas[ii*kk+1], K.y])
        # Position vector AB in fixed reference of frame
        rAB = G.dcm(H) * (G.dcm(K) * mat.Matrix([0, 0, act_spc[ii]]))
        # Contruct all parameters for each limb
        s_limb = []
        for jj, item in enumerate(_s):
            if jj == 1:
                item = G.dcm(K) * item
            elif jj in [2, 3, 4, 5]:
                item = G.dcm(N) * item
            item = G.dcm(H) * item
            _r = G.dcm(H) * mat.Matrix([R, 0, 0])
            if jj in [3, 4, 5]:
                _r += rAB           
            s_limb.append([item, _r, _h[jj]])
        list_of_parameters.append(s_limb) 
    return list_of_parameters


**The first to fourth step:**

In [23]:
Mobility_PM_6UPS = s4m.ManipulatorsMobility(PM_6UPS(), "6-UPS Parallel Manipulators")

In [24]:
Mobility_PM_6UPS.Mobility

{'Number': 6,
 'Type': '3T3R',
 'Direction': ['Tx', 'Ty', 'Tz', 'Rx', 'Ry', 'Rz']}

**Displaying the result from the first to fourth step:** 

In [25]:
all_screw_system(Mobility_PM_6UPS)

Limb Twist System
Limb:  1


⎡⎡1⎤  ⎡           0           ⎤  ⎡                    0                     ⎤ 
⎢⎢ ⎥  ⎢                       ⎥  ⎢                                          ⎥ 
⎢⎢0⎥  ⎢ cos(\theta_{{1}{1}})  ⎥  ⎢                    0                     ⎥ 
⎢⎢ ⎥  ⎢                       ⎥  ⎢                                          ⎥ 
⎢⎢0⎥  ⎢ sin(\theta_{{1}{1}})  ⎥  ⎢                    0                     ⎥ 
⎢⎢ ⎥, ⎢                       ⎥, ⎢                                          ⎥,
⎢⎢0⎥  ⎢           0           ⎥  ⎢           sin(\theta_{{1}{2}})           ⎥ 
⎢⎢ ⎥  ⎢                       ⎥  ⎢                                          ⎥ 
⎢⎢0⎥  ⎢-R⋅sin(\theta_{{1}{1}})⎥  ⎢-sin(\theta_{{1}{1}})⋅cos(\theta_{{1}{2}})⎥ 
⎢⎢ ⎥  ⎢                       ⎥  ⎢                                          ⎥ 
⎣⎣0⎦  ⎣R⋅cos(\theta_{{1}{1}}) ⎦  ⎣cos(\theta_{{1}{1}})⋅cos(\theta_{{1}{2}}) ⎦ 

 ⎡                           cos(\theta_{{1}{2}})                           ⎤ 
 ⎢                                                 

Limb:  2


⎡       ⎡-√3⋅cos(\theta_{{2}{1}}) ⎤                                           
⎢       ⎢─────────────────────────⎥                                           
⎢       ⎢            2            ⎥  ⎡                                  0     
⎢⎡1/2⎤  ⎢                         ⎥  ⎢                                        
⎢⎢   ⎥  ⎢  cos(\theta_{{2}{1}})   ⎥  ⎢                                  0     
⎢⎢√3 ⎥  ⎢  ────────────────────   ⎥  ⎢                                        
⎢⎢── ⎥  ⎢           2             ⎥  ⎢                                  0     
⎢⎢2  ⎥  ⎢                         ⎥  ⎢                                        
⎢⎢   ⎥  ⎢  sin(\theta_{{2}{1}})   ⎥  ⎢ √3⋅sin(\theta_{{2}{1}})⋅cos(\theta_{{2}
⎢⎢ 0 ⎥, ⎢                         ⎥, ⎢ ───────────────────────────────────────
⎢⎢   ⎥  ⎢√3⋅R⋅sin(\theta_{{2}{1}})⎥  ⎢                      2                 
⎢⎢ 0 ⎥  ⎢─────────────────────────⎥  ⎢                                        
⎢⎢   ⎥  ⎢            2            ⎥  ⎢  sin(\theta_{

Limb:  3


...

[]



Platform Wrench System


[]



Platform Twist System


⎡⎡0⎤  ⎡0⎤  ⎡0⎤  ⎡1⎤  ⎡0⎤  ⎡0⎤⎤
⎢⎢ ⎥  ⎢ ⎥  ⎢ ⎥  ⎢ ⎥  ⎢ ⎥  ⎢ ⎥⎥
⎢⎢0⎥  ⎢0⎥  ⎢0⎥  ⎢0⎥  ⎢1⎥  ⎢0⎥⎥
⎢⎢ ⎥  ⎢ ⎥  ⎢ ⎥  ⎢ ⎥  ⎢ ⎥  ⎢ ⎥⎥
⎢⎢0⎥  ⎢0⎥  ⎢0⎥  ⎢0⎥  ⎢0⎥  ⎢1⎥⎥
⎢⎢ ⎥, ⎢ ⎥, ⎢ ⎥, ⎢ ⎥, ⎢ ⎥, ⎢ ⎥⎥
⎢⎢1⎥  ⎢0⎥  ⎢0⎥  ⎢0⎥  ⎢0⎥  ⎢0⎥⎥
⎢⎢ ⎥  ⎢ ⎥  ⎢ ⎥  ⎢ ⎥  ⎢ ⎥  ⎢ ⎥⎥
⎢⎢0⎥  ⎢1⎥  ⎢0⎥  ⎢0⎥  ⎢0⎥  ⎢0⎥⎥
⎢⎢ ⎥  ⎢ ⎥  ⎢ ⎥  ⎢ ⎥  ⎢ ⎥  ⎢ ⎥⎥
⎣⎣0⎦  ⎣0⎦  ⎣1⎦  ⎣0⎦  ⎣0⎦  ⎣0⎦⎦

## 4. Computation Time for Each PM

**The 3-PRRR Parallel Manipulator**

In [26]:
%%timeit -n 1 -r 10
s4m.ManipulatorsMobility(PM_3PRRR())

1.16 s ± 45.1 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


**The 3-PR(Pa)R Parallel Manipulator**

In [27]:
%%timeit -n 1 -r 10
s4m.ManipulatorsMobility(PM_3PRPaR())

5.24 s ± 147 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


**The 4-PRRU Parallel Manipulator**

In [28]:
%%timeit -n 1 -r 10
s4m.ManipulatorsMobility(PM_4PRRU())

9.01 s ± 329 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


**The 6-UPS Parallel Manipulator**

In [29]:
%%timeit -n 1 -r 10
s4m.ManipulatorsMobility(PM_6UPS())

25.9 s ± 418 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


---